## Importing the Libraries

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
dataset = #Dataset Path for face recognition

In [ ]:
only_face = #Path to save deted face from the Dataset

## Use Either of the way to detect the faces

In [ ]:
from PIL import Image, ImageOps
import numpy as np
cascPath = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

for folder in os.listdir(dataset):
  count = 0
  print(folder)
  for img in os.listdir(dataset + folder):
      im1 = Image.open(dataset + folder + '/' + img)
      im2 = np.asarray(im1)
      faces = faceCascade.detectMultiScale(im2,
                                            scaleFactor=1.1,
                                            minNeighbors=5,
                                            minSize=(60, 60),
                                            flags=cv2.CASCADE_SCALE_IMAGE)

      for (x, y, w, h) in faces:
            #face_frame = Image.fromarray(im2)
            face_frame = im2[y:y+h,x:x+w]
            face_frame = Image.fromarray(face_frame)
            face_frame.save(only_face + folder + '/' + str(count) + '.jpg')
            print(img) 
      count += 1      

In [ ]:
import cv2

cascPath = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

for folder in os.listdir(dataset):
  count = 0
  print(folder)
  for img in os.listdir(dataset + folder):
    print(img)
    #grey_img = cv2.imread(dataset + folder + '/' + img, cv2.IMREAD_GRAYSCALE)
    grey_img = cv2.imread(dataset + folder + '/' + img)
    faces = faceCascade.detectMultiScale(grey_img,
                                      scaleFactor=1.1,
                                      minNeighbors=5,
                                      minSize=(60, 60),
                                      flags=cv2.CASCADE_SCALE_IMAGE)
    for (x, y, w, h) in faces:
      face_frame = grey_img[y:y+h,x:x+w]
      face_frame1 = cv2.cvtColor(face_frame, cv2.COLOR_GRAY2BGR )
      cv2.imwrite('onlyface1/' + folder + '/' + str(count) + '.jpg',face_frame1)
    count += 1
  

## If you have Validation Dataset remove the Validation split here

In [ ]:
data_datagen = ImageDataGenerator(
    rescale = 1./255,
    #shear_range=0.1,
    brightness_range=[0.1,0.2],
    #height_shift_range=0.5,
    #width_shift_range=0.5,
    rotation_range=0.10,
    horizontal_flip = True,
    validation_split=0.29  
)

In [ ]:
train_generator = data_datagen.flow_from_directory(
    only_face,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical',
    #color_mode = 'grayscale',
    subset = 'training')

In [ ]:
test_generator = data_datagen.flow_from_directory(
    only_face,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical',
    #color_mode = 'grayscale',
    subset = 'validation')

In [ ]:
train_generator.class_indices

In [ ]:
train_generator[0][0].shape

## Train Using Inception V3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
import tensorflow
from time import time
#from tensorflow.python.keras.callbacks import TensorBoard

#tensorboard = TensorBoard(log_dir="logs\{}".format(time()))

x = Flatten()(base_model.output)
x = Dense(1024, activation='relu',
    kernel_regularizer= tensorflow.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=tensorflow.keras.regularizers.l2(1e-4),
    activity_regularizer=tensorflow.keras.regularizers.l2(1e-5))(x)
x = Dropout(0.2)(x)
x = Dense(3, activation='softmax')(x)

model = Model(base_model.input, x)

model.compile(optimizer = Adam(lr=0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
inc_history = model.fit(train_generator,
                                  validation_data = test_generator,
                                  #steps_per_epoch = 100,
                                  #callbacks=[tensorboard],
                                  epochs = 20)

In [ ]:
acc      = inc_history.history['accuracy']
val_acc  = inc_history.history['val_accuracy']
loss     = inc_history.history['loss']
val_loss = inc_history.history['val_loss']

epochs   = range(len(acc)) # Get number of epochs

plt.plot  (epochs, acc)
plt.plot  (epochs, val_acc)
plt.title ('Training and validation accuracy')
plt.figure()
plt.plot  (epochs, loss)
plt.plot  (epochs, val_loss)
plt.title ('Training and validation loss')

In [ ]:
model.save('main_model.h5')

## Sample Prediction and testing

In [ ]:
from PIL import Image, ImageOps
import numpy as np
import keras
import tensorflow


np.set_printoptions(suppress=True)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
image = Image.open('dataset\Deepak\IMG_20201116_172722.jpg')#.convert('L')
print(image.size)
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)
print(image.size)
image_array = np.asarray(image)
print(image_array.size)
image.show()
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
data[0] = normalized_image_array
model = tensorflow.keras.models.load_model("normal_main_model_150.h5")
prediction = model.predict(data)
print(prediction)